In [1]:
#import the key libraries
from bs4 import BeautifulSoup
from datetime import datetime

import requests
import csv
import re
import time
import datetime

#need to store the data in a dataframe
import pandas as pd 

In [2]:
#initialize the variables
debugFlag = 0
uniqueid = 0

gowers_article_list = []
tao_article_list = []

In [3]:
#create the empty dataframe for initial posts and comments
polymath_BERT = pd.DataFrame(columns = ['post_source','post_url','post_timestamp', 'comment_timestamp','post_title','counter', 'post_author', 'post_word_count','post_text'])
print(polymath_BERT)   

Empty DataFrame
Columns: [post_source, post_url, post_timestamp, comment_timestamp, post_title, counter, post_author, post_word_count, post_text]
Index: []


In [6]:
### Reads from csv source and appends to article link list
#changes from source code:
# switched to 'r' so it wasn't a binary file; 
# added encoding - the utf-8-sig removes the '\ufeff' that was showing up at the beginning of the first row
with open('/Users/jennaolsen/Desktop/mothership-gowers-articles-list.csv', 'r', encoding="unicode_escape") as f:
	reader = csv.reader(f)
	for row in reader:
		gowers_article_list.append(row)

In [7]:
### Reads from csv source and appends to article link list
#changes from source code:
# switched to 'r' so it wasn't a binary file; 
# added encoding - the utf-8-sig removes the '\ufeff' that was showing up at the beginning of the first row
with open('/Users/jennaolsen/Desktop/mothership-tao-articles-list.csv', 'r', encoding="unicode_escape") as f:
	reader = csv.reader(f)
	for row in reader:
		tao_article_list.append(row)

In [ ]:
# ###  Checks the content of the article_link_list
# for row in gowers_article_list:
# 	print((row))
# print(len(gowers_article_list))

In [8]:
# pulling data from the gowers primary posts
for article_link in gowers_article_list:

	# Masking nature of the bot
	if int(datetime.datetime.now().second) % 3 == 0:
		user_agent = 'Mozilla/5 (Solaris 10) Gecko'
	elif int(datetime.datetime.now().second) % 5 == 0:
		user_agent = 'Mozilla/4.0 (compatible; MSIE 10.0; Windows NT 6.1; Trident/5.0)'
	else:
		user_agent = 'Mozilla/5.0 (compatible; MSIE 5.5; Windows NT)'

	headers = {'User-Agent': user_agent}

	time.sleep(1.4)

# 	print "\n\n=====Task ID: " + str(uniqueid) + "=====\n"

	article_link = ''.join(article_link)
	print ("Now scraping: " + article_link + "...")
	r = requests.get(article_link)
	soupData = BeautifulSoup(r.text)

	### Scraping the title - had to remove the encoding because it killed concate operation in the next line
	title = soupData.find('h2').text
# 	title = title.encode('utf-8-sig').strip()
# 	print ("Title: " + title)

# <meta property="article:published_time" content="2009-02-01T12:43:20+00:00" />

	### Scraping the timestamp
	timestamp = soupData.find("meta", property="article:published_time")
	ts = timestamp["content"]
	print ("Published Time: " + ts.replace('T',' ').replace('+00:00',''))

	### Scraping the entry
	entry = soupData.find_all("div", {"class": "entry"})
	post_content = str(entry)
	### Parsing the text into a beautfulsoup item
	parsed_post_content = BeautifulSoup(post_content)
	### Find all paragraphs/blog content
	content_tokens = parsed_post_content.findAll('p')
	### The last 2 paragraphs are unwanted
# 	advert_text = content_tokens[len(content_tokens)-1]
	### parse to string form
	advert_text = str(content_tokens)
	### parse to string form as well
# 	post_content = parsed_post_content.text
	post_content = post_content.replace(advert_text, "")
	### To remove all newlines
	post_content = post_content.replace("\n", "").replace("\r", "").strip()
	### To remove all tags
	post_content = BeautifulSoup(post_content, "lxml").text
    ### To remove the garbage left at the end of the text
	post_content = post_content.replace("Share this:TwitterFacebookLike this:Like Loading...Related", "").strip()#.encode('utf-8')
    
    ###word count for the post 
	word_count = len(post_content.split())
    
    ###Add the new row to the dataframe 
	new_row = {'post_source':'Gowers','post_url':article_link,'post_timestamp':ts,'post_title':title,'counter':'0', 'post_author':'gowers', 'post_word_count':word_count, 'post_text':post_content}
	polymath_BERT = polymath_BERT.append(new_row, ignore_index=True)                                

Now scraping: book    mark    8   8   |           tî`îÅA       T               Users   ...


InvalidURL: Failed to parse: book    mark    8   8   |           tî`îÅA       T               Users   

In [ ]:
# pulling comments from the various articles
# move into main section once it is working
for article_link in gowers_article_list:

	# Masking nature of the bot
	if int(datetime.datetime.now().second) % 3 == 0:
		user_agent = 'Mozilla/5 (Solaris 10) Gecko'
	elif int(datetime.datetime.now().second) % 5 == 0:
		user_agent = 'Mozilla/4.0 (compatible; MSIE 10.0; Windows NT 6.1; Trident/5.0)'
	else:
		user_agent = 'Mozilla/5.0 (compatible; MSIE 5.5; Windows NT)'

	headers = {'User-Agent': user_agent}

	time.sleep(1.4)

	article_link = ''.join(article_link)
	print ("Now scraping: " + article_link + "...") # the "post_url" field
	r = requests.get(article_link)
	soupData = BeautifulSoup(r.text)
    
### Scraping for comments
	comments_count = soupData.find_all("li", attrs={"id" : True})
	for cc in comments_count:
		ident = cc["id"].replace("comment-","C").strip()
# 		print(ident) #this will be the "counter" field
        
		time_string = cc.find("small").text.replace(" | Reply ","").strip()
# 		print(time_string) # the "post_timestamp" field
        
		the_comment = str(cc.findAll("p"))
		the_comment = BeautifulSoup(the_comment, "lxml").text
# 		print(the_comment) # the "post_text" field

		author = cc.find("cite").string
# 		print(author) # the "post_text" field

		word_count = len(str(the_comment).split())

        ###insert the rw into the dataframe
		new_row = {'post_source':'Gowers','post_url':article_link,'comment_timestamp':time_string,'post_title':'','counter':ident, 'post_author':author, 'post_word_count':word_count, 'post_text':the_comment}
		polymath_BERT = polymath_BERT.append(new_row, ignore_index=True)

In [ ]:
###  Checks the content of the article_link_list
for row in tao_article_list:
	print((row))
print(len(tao_article_list))

In [ ]:
# pulling data from the various articles
for article_link in tao_article_list:

	# Masking nature of the bot
	if int(datetime.datetime.now().second) % 3 == 0:
		user_agent = 'Mozilla/5 (Solaris 10) Gecko'
	elif int(datetime.datetime.now().second) % 5 == 0:
		user_agent = 'Mozilla/4.0 (compatible; MSIE 10.0; Windows NT 6.1; Trident/5.0)'
	else:
		user_agent = 'Mozilla/5.0 (compatible; MSIE 5.5; Windows NT)'

	headers = {'User-Agent': user_agent}

	time.sleep(1.4)

	article_link = ''.join(article_link)
	print ("Now scraping: " + article_link + "...")
	r = requests.get(article_link)
	soupData = BeautifulSoup(r.text)

	### Scraping the title - had to remove the encoding because it killed concate operation in the next line
	title = soupData.find('title').text.replace(" | What's new","")
# 	print ("Title: " + title)

	### Scraping the timestamp
	timestamp = soupData.find("meta", property="article:published_time")
# 	print ("Published Time: " + timestamp["content"])
	ts = timestamp["content"]

	### Scraping the entry
	entry = soupData.find_all("div", {"class": "post-content"})
	post_content = str(entry)
	### Parsing the text into a beautfulsoup item
	parsed_post_content = BeautifulSoup(post_content)
	### parse to string form
	advert_text = parsed_post_content.text
	## parse to string form as well
	post_content = advert_text
    ### To remove the garbage left at the end of the text
	post_content = post_content.replace("Further articles on this project are collected at this page.", "").replace("<\b>", "")
	post_content = post_content.replace("\n\nShare this:PrintEmailMoreTwitterFacebookRedditPinterestLike this:Like Loading... ", "").strip().encode('utf-8')
# 	print (post_content)
   
	## parse to string form as well
	post_content = BeautifulSoup(post_content, "lxml").text
   
	##word count of post
	word_count = len(post_content.split())
    
#     ###Add the new row to the dataframe 
	new_row = {'post_source':'Tao','post_url':article_link,'post_timestamp':ts,'post_title':title,'counter':'0', 'post_author':'Tao', 'post_word_count':word_count, 'post_text':post_content}
	polymath_BERT = polymath_BERT.append(new_row, ignore_index=True)   

In [ ]:
for article_link in tao_article_list:

	# Masking nature of the bot
	if int(datetime.datetime.now().second) % 3 == 0:
		user_agent = 'Mozilla/5 (Solaris 10) Gecko'
	elif int(datetime.datetime.now().second) % 5 == 0:
		user_agent = 'Mozilla/4.0 (compatible; MSIE 10.0; Windows NT 6.1; Trident/5.0)'
	else:
		user_agent = 'Mozilla/5.0 (compatible; MSIE 5.5; Windows NT)'

	headers = {'User-Agent': user_agent}

	time.sleep(1.4)

	comment_count = 0
    
	article_link = ''.join(article_link)
	print ("Now scraping: " + article_link + "...") # the "post_url" field
	r = requests.get(article_link)
	soupData = BeautifulSoup(r.text)
    
### Scraping for comments
### Comments for Tao show up with two different tags (so far) - this structure allows find_all to find both of them  'div[class*="listing-col-"]'
	comments_count = soupData.select('div[class*="-comment"]')
	for cc in comments_count:
        #Adding the CTao in front of the number to avoid duplication with Gowers blog posts
		ident = cc["id"].replace("comment-","CTao_").strip()
# 		print(ident) #this will be the "counter" field


        #Temporary to identify the posts where I am missing some comments
		comment_count+=1
# 		print(ident) #this will be the "counter" field

        
		time_string = cc.find("p", attrs={"class":"comment-permalink"}).text.strip()
# 		print(time_string) # the "post_timestamp" field
        
		the_comment = str(cc.find("div", attrs={"class":"comment-content"}).findAll("p"))
		the_comment = BeautifulSoup(the_comment, "lxml").text
# 		print(the_comment) # the "post_text" field
        
		the_author = cc.find("p", attrs={"class":"comment-author"}).find("strong").string
# 		print(the_author) # the "post_text" field
        
		word_count = len(str(the_comment).split())

        ###insert the rw into the dataframe
		new_row = {'post_source':'Tao','post_url':article_link,'comment_timestamp':time_string,'post_title':'','counter':ident, 'post_author':the_author, 'post_word_count':word_count, 'post_text':the_comment}
		polymath_BERT = polymath_BERT.append(new_row, ignore_index=True)

In [ ]:
#ready to set up for the BERT analysis now :) 

In [ ]:
### checking for the full count
### 1543 comments and 20 posts from Gowers
### 888 comments and 14 posts from Tao
print(polymath_BERT)

In [ ]:
# polymath_BERT.to_excel('polymath_BERT_wordCounts.xlsx')